#### 제2유형. 타이타닉 생존자 분류하기

1. 라이브러리 및 데이터 확인

2. 데이터 탐색(EDA)

3. 데이터 전처리 및 분리 (결측치, 이상치 대체)

4. 모델링 및 성능 평가

5. 예측값 제출

In [490]:
# 라이브러리 및 데이터 확인

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# 분류 성능
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

# 회귀 성능
from sklearn.metrics import r2_score, mean_squared_error

# 출력 범위 조정
pd.options.display.max_columns = 999

In [491]:
# 데이터셋 불러오기
import seaborn as sns

df = sns.load_dataset('titanic')

x = df.drop('survived', axis=1)
y = df['survived']

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=2023)

cust_id = pd.DataFrame(x_test.reset_index())['index']

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)


y_train.columns = ['target']
# y_train.rename(columns={'survived':'target'}, inplace=True)
y_test.columns = ['target']

# y_train.head()
# x_test.head()
# x_train.head()

In [492]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

# x_train : age, embarked, deck, embark_town 결측치 확인
# x_test : age, deck 결측치 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 3 to 608
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   pclass       712 non-null    int64   
 1   sex          712 non-null    object  
 2   age          579 non-null    float64 
 3   sibsp        712 non-null    int64   
 4   parch        712 non-null    int64   
 5   fare         712 non-null    float64 
 6   embarked     710 non-null    object  
 7   class        712 non-null    category
 8   who          712 non-null    object  
 9   adult_male   712 non-null    bool    
 10  deck         164 non-null    category
 11  embark_town  710 non-null    object  
 12  alive        712 non-null    object  
 13  alone        712 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(3), object(5)
memory usage: 64.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 800 to 410
Data columns (total 14 columns):
 #   Colum

In [493]:
print(x_train.describe().T)
print(x_test.describe().T)
print(y_train.describe().T)

        count       mean        std   min      25%      50%     75%       max
pclass  712.0   2.307584   0.834926  1.00   2.0000   3.0000   3.000    3.0000
age     579.0  29.479568  14.355304  0.42  20.0000  28.0000  38.000   74.0000
sibsp   712.0   0.518258   1.094522  0.00   0.0000   0.0000   1.000    8.0000
parch   712.0   0.372191   0.792341  0.00   0.0000   0.0000   0.000    6.0000
fare    712.0  31.741836  45.403910  0.00   7.8958  14.4542  31.275  512.3292
        count       mean        std  min     25%   50%      75%       max
pclass  179.0   2.312849   0.842950  1.0   2.000   3.0   3.0000    3.0000
age     135.0  30.640741  15.258427  1.0  22.000  29.0  39.0000   80.0000
sibsp   179.0   0.541899   1.137797  0.0   0.000   0.0   1.0000    8.0000
parch   179.0   0.418994   0.859760  0.0   0.000   0.0   0.0000    5.0000
fare    179.0  34.043364  64.097184  0.0   7.925  14.5  30.2854  512.3292
        count      mean       std  min  25%  50%  75%  max
target  712.0  0.383427  0.48

In [494]:
print(y_train.value_counts())

target
0         439
1         273
dtype: int64


In [495]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())

pclass           0
sex              0
age            133
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           548
embark_town      2
alive            0
alone            0
dtype: int64
pclass           0
sex              0
age             44
sibsp            0
parch            0
fare             0
embarked         0
class            0
who              0
adult_male       0
deck           140
embark_town      0
alive            0
alone            0
dtype: int64


In [496]:
# 방법2. 카테고리 추가 및 결측치 대체

# x_train['deck'] = x_train['deck'].cat.add_categories("N")
# x_test['deck'] = x_test['deck'].cat.add_categories("N")

# x_train['deck'] = x_train['deck'].fillna("N")
# x_test['deck'] = x_test['deck'].fillna("N")

# print(x_train['deck'].value_counts())
# print(x_test['deck'].value_counts())


In [497]:
# 방법1. deck 컬럼 제거
x_train.drop(columns=['deck'], axis=1, inplace=True)
x_test.drop(columns=['deck'], axis=1, inplace=True)

print(x_train.shape)
print(x_test.shape)

(712, 13)
(179, 13)


In [498]:
# age는 median 값으로 대체
age_median = x['age'].median()
print(age_median)

x_train['age'] = x_train['age'].fillna(age_median)
x_test['age'] = x_test['age'].fillna(age_median)

28.0


In [499]:
# embark_town 최빈값으로 대체

town_mode = x['embark_town'].mode()[0]
print(town_mode)

x_train['embark_town'] = x_train['embark_town'].fillna(town_mode)

Southampton


In [500]:
# embarked 최빈값으로 대체

embarked_mode = x['embarked'].mode()[0]
print(embarked_mode)

x_train['embarked'] = x_train['embarked'].fillna(embarked_mode)

S


In [501]:
# 전처리 결과 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())

pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64


In [502]:
# 중복되는 변수 drop 처리
x_train.drop(columns=['class','embark_town','alive'],axis=1, inplace = True)
x_test.drop(columns=['class','embark_town','alive'], axis=1, inplace = True)

In [503]:
# 범주형 데이터 원핫인코딩 적용
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

print(x_train.shape)
print(x_test.shape)

# print(x_train.columns)

(712, 15)
(179, 15)


In [504]:
# 모델링 및 성능평가

model = RandomForestClassifier()

model.fit(x_train, y_train['target'])

RandomForestClassifier()

In [505]:
# 모델 검증

X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train['target'], stratify=y_train['target'], test_size=0.2, random_state=2023)

# print(Y_val.shape)

model_val = RandomForestClassifier()
model_val.fit(X_train, Y_train)

y_pred = model_val.predict(X_val)

accuracy = accuracy_score(Y_val, y_pred)
f1_s = f1_score(Y_val, y_pred, average='macro')
roc_auc = roc_auc_score(Y_val, y_pred)

print(accuracy)
print(f1_s)
print(roc_auc)

0.8321678321678322
0.8238916256157636
0.8261363636363637


In [506]:
# 예측값 도출 및 결과 제출

y_result = model.predict(x_test)

result = pd.DataFrame({"cust_id":cust_id, "result" : y_result})

# print(result)

result.to_csv('test_result.csv', index=False)

In [507]:
print(pd.read_csv('test_result.csv'))

     cust_id  result
0        800       0
1        341       1
2        413       0
3        575       0
4        202       0
..       ...     ...
174      402       0
175       70       0
176       53       1
177      743       0
178      410       0

[179 rows x 2 columns]


In [508]:
# 최종 성능 확인

accuracy = accuracy_score(y_test, y_result)
f1 = f1_score(y_test, y_result, average='macro')
roc_auc = roc_auc_score(y_test, y_result)

print(accuracy, f1, roc_auc)

0.7877094972067039 0.7719898096004291 0.7678524374176547


In [510]:
# 원핫인코딩 후 feature 변수가 다른 경우
# reindex 활용

# x_train = x_train.reindex(columns = x_test.columns, fill_value=0)

In [511]:
# 참고사항! 혼동행렬
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_result)

print(cm)

# 읽는 방법
#            예측
#           0    1
# 실제  0  TN   FP
#       1 FN   TP

[[94 16]
 [22 47]]
